In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [2]:
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')
df_sample = pd.read_csv('sample_submission.csv')

In [3]:
df_test.head()

,seq_id,protein_sequence,pH,data_source
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes


In [4]:
df_train.head()

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31390 entries, 0 to 31389
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seq_id            31390 non-null  int64  
 1   protein_sequence  31390 non-null  object 
 2   pH                31104 non-null  float64
 3   data_source       28043 non-null  object 
 4   tm                31390 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.2+ MB


In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2413 entries, 0 to 2412
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   seq_id            2413 non-null   int64 
 1   protein_sequence  2413 non-null   object
 2   pH                2413 non-null   int64 
 3   data_source       2413 non-null   object
dtypes: int64(2), object(2)
memory usage: 75.5+ KB


In [7]:
df_train.isnull().sum()

seq_id                 0
protein_sequence       0
pH                   286
data_source         3347
tm                     0
dtype: int64

In [8]:
df_test.isnull().sum()

seq_id              0
protein_sequence    0
pH                  0
data_source         0
dtype: int64

In [9]:
df_train.shape,df_test.shape

((31390, 5), (2413, 4))

In [10]:
cat_columns = df_train.select_dtypes(['object']).columns

In [11]:
df_train[cat_columns] = df_train[cat_columns].apply(lambda x: pd.factorize(x)[0])
cat_columns

Index(['protein_sequence', 'data_source'], dtype='object')

In [12]:
df_train

,seq_id,protein_sequence,pH,data_source,tm
0,0,0,7.0,0,75.7
1,1,1,7.0,0,50.5
2,2,2,7.0,0,40.5
3,3,3,7.0,0,47.2
4,4,4,7.0,0,49.5
...,...,...,...,...,...
31385,31385,28976,7.0,0,51.8
31386,31386,28977,7.0,0,37.2
31387,31387,28978,7.0,0,64.6
31388,31388,28979,7.0,0,50.7


In [13]:
df_t = df_train.drop(['data_source','seq_id'],axis=1)

In [14]:
df_t.shape

(31390, 3)

In [15]:
cate_columns = df_test.select_dtypes(['object']).columns
cate_columns

Index(['protein_sequence', 'data_source'], dtype='object')

In [16]:
df_test[cate_columns] = df_test[cate_columns].apply(lambda x: pd.factorize(x)[0])

In [17]:
df_tes = df_test.drop(['data_source','seq_id'],axis=1)

In [18]:
Y = df_t['tm']
X = df_t.drop(['tm'],axis=1)

In [19]:
Y.value_counts()

25.0     1472
20.0      561
50.0      200
45.0      178
49.7      176
         ... 
98.1        1
97.6        1
92.7        1
1.2         1
103.0       1
Name: tm, Length: 770, dtype: int64

In [20]:
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [21]:
# Splitting
train_X, test_X, train_y, test_y = train_test_split(X, Y,test_size = 0.3)

In [22]:
import catboost as cb
import lightgbm as ltb


#xgb_r = xg.XGBRegressor(learning_rate=0.01, max_depth=10, n_estimators=1500)
#model1 = cb.CatBoostRegressor(learning_rate=0.02,loss_function='RMSE',iterations= 1800,depth= 6,l2_leaf_reg=2,) 
model2 = ltb.LGBMRegressor(learning_rate=0.2,max_depth= 10,
                           boosting_type='gbdt' ,n_estimators =800, metric = ['l1','l2'], num_leaves = 128,  
    max_bin = 512) 


# Fitting the model
model2.fit(train_X, train_y)

ModuleNotFoundError: No module named 'catboost'

In [ ]:
%pip install catboost

In [ ]:
from sklearn.metrics import r2_score

pred = model2.predict(test_X)
rmse = (np.sqrt(mean_squared_error(test_y, pred)))
r2 = r2_score(test_y, pred)
print("Testing performance")
print("RMSE: {:.2f}".format(rmse))
print("R2: {:.2f}".format(r2))

In [ ]:
y_pred = model2.predict(df_tes)
y_pred = y_pred.T
y_pred

In [ ]:
df_sample["tm"] = y_pred 
df_sample.to_csv("LGBMsubmission.csv", index=False)

In [ ]:
sub = pd.read_csv('/kaggle/working/LGBMsubmission.csv')
sub.head()